<img src="https://media-exp1.licdn.com/dms/image/C560BAQHgXkBxAhPV2Q/company-logo_200_200/0?e=2159024400&v=beta&t=1tsomaFfzFF5LcVhqwqP3sJtGMb8qC_cDGzF5BC6tfg" width="60" align = "left" alt="MEF logo"  />

#### <font color='grb({0},{192},{0)'>$$\large\text{Descarga de datos}$$</font>

### **1. Consulta Amigable de Gastos**

In [ ]:
import pandas as pd   #paquete para el manejo de tablas o DataFrames
from tqdm import tqdm #Muestra la barra de progreso de una iteración 
from tqdm.notebook import tqdm_notebook 
import Scrapper_Consulta_Amigable #Automatiza la descarga de datos en la Consulta Amigable del MEF
from Scrapper_Consulta_Amigable import CONSULTA #Importa la clase CONSULTA
import time           #Importa la libreria que permite registrar tiempo
from joblib import Parallel, delayed #Libreria que permite paralelizar varias tareas

#### 1.1. Gobierno nacional

In [ ]:
def CONSULTA_GN(x):
    '''
    Función que descarga proyectos del gobierno nacional 
    '''
    import pandas as pd                    #Librería para manejo de DataFrames
    import Scrapper_Consulta_Amigable      #Automatiza la descarga de datos en la Consulta Amigable del MEF
    from Scrapper_Consulta_Amigable import CONSULTA
    BP = CONSULTA()                        #Abre el navegador de Chrome y se redirige a la página de la Consulta Amigable
    BP.AÑO(x)                              #Selecciona año  
    BP.ACTIVIDADES('PROYECTO')             #Selecciona solo proyectos
    BP.SELECCION('NIVEL','E')              #Selecciona nivel de gobierno nacional
    BP.SELECCION('SECTOR')                 #Filtra por sector 
    nombres = BP.nombres()                 #Extrae el nombre del sector
    filas = BP.indices()                   #Extrae el código identificador de cada sector  
    contenedor = []                        #Almacenará los datos recabados
    for x,y in zip(filas,nombres):         #Se itera para cada sector hallado en el año
        BP.SELECCION('SECTOR',x)           #Se selecciona un sector
        #Extrae data anual de proyectos y agrega columna con nombre de sector y código 
        contenedor.append(BP.SELECCION('PROYECTO','TODO_A',{'SECTOR':x, 'Nomb_Sect':y}))
        BP.retroceder()                    #Regresa a la ventana anterior para seleccionar otro sector
    BP.CERRAR()                            #Cierra el navegador al terminar la iteración 
    proy_inversion = pd.concat(contenedor) #Se concatenan las tablas contenidas en la lista
    proy_inversion = proy_inversion.reset_index(drop=True) #Vuelve al índice por default
    return proy_inversion                  #Retorna la tabla de proyectos como resultado

In [ ]:
start = time.time()

#Extrae los datos de de proyectos del 2004 al 2022 (realiza cuatro búsquedas al mismo tiempo)
GN = Parallel(n_jobs=4)(delayed(CONSULTA_GN)(x) for x in tqdm(range(2004,2022)))

end = time.time()
print(end - start)

#### 1.2. Gobiernos regionales 

In [ ]:
def CONSULTA_GR(x):
    '''
    Función que descarga proyectos del gobierno regional por pliego 
    '''
    import pandas as pd                    #Librería para manejo de DataFrames
    import Scrapper_Consulta_Amigable      #Automatiza la descarga de datos en la Consulta Amigable del MEF
    from Scrapper_Consulta_Amigable import CONSULTA
    BP = CONSULTA()                        #Abre el navegador de Chrome y se redirige a la página de la Consulta Amigable
    BP.AÑO(x)                              #Selecciona año  
    BP.ACTIVIDADES('PROYECTO')             #Selecciona solo proyectos
    BP.SELECCION('NIVEL','R')              #Selecciona nivel de gobierno nacional
    BP.SELECCION('SECTOR','99')            #Filtro por sector
    BP.SELECCION('PLIEGO')                 #Filtro por  pliego
    nombres = BP.nombres()                 #Extrae nombre del pliego
    filas = BP.indices()                   #Extrae el código identificador de cada sector  
    contenedor = []                        #Almacenará los datos recabados
    for x,y in zip(filas,nombres):         #Se itera para cada pliego hallado en el año
        BP.SELECCION('PLIEGO',x)           #Se selecciona un pliego
        #Se extraen los datos de proyectos y se agrega columna con nombre de pliego y código
        contenedor.append(BP.SELECCION('PROYECTO','TODO_A',{'PLIEGO':x, 'Nomb_Sect':y}))
        BP.retroceder()                    #Regresa a la ventana anterior para seleccionar otro sector
    BP.CERRAR()                            #Cierra el navegador al terminar la iteración 
    proy_inversion = pd.concat(contenedor) #Se concatenan las tablas contenidas en la lista
    proy_inversion = proy_inversion.reset_index(drop=True)
    return proy_inversion

In [ ]:
start = time.time()

#Extrae los datos de de proyectos del 2004 al 2022 (realiza cuatro búsquedas al mismo tiempo)
GR = Parallel(n_jobs=4)(delayed(CONSULTA_GR)(x) for x in tqdm(range(2004,2022)))

end = time.time()
print(end - start)

#### 1.3. Gobiernos locales

In [ ]:
def CONSULTA_GL(x):
    '''
    Función que descarga proyectos de gobiernos locales por municipalidad
    '''
    import pandas as pd                    #Librería para manejo de DataFrames
    import Scrapper_Consulta_Amigable      #Automatiza la descarga de datos en la Consulta Amigable del MEF
    from Scrapper_Consulta_Amigable import CONSULTA
    BP = CONSULTA()                        #Abre el navegador de Chrome y se redirige a la página de la Consulta Amigable
    BP.AÑO(x)                              #Selecciona año  
    BP.ACTIVIDADES('PROYECTO')             #Selecciona solo proyectos
    BP.SELECCION('NIVEL','M')              #Filtra por nivel de gobierno y selecciona GL
    BP.SELECCION('GOB.LOC./MANCOM.','M')   #Filtra por GOB.LOC./MANCOM. y selecciona GL
    BP.SELECCION('DEPARTAMENTO')           #Filtra por departamento
    dep = BP.nombres()                     #Extrae nombre de departamento
    filas = BP.indices()                   #Extrae código de departamento
    contenedor = []                        #Lista para guardar DataFrame
    for x,y in tqdm(zip(filas,dep)):       #Itera para cada departamento
        BP.SELECCION('DEPARTAMENTO',x)     #Selecciona un departamento específico
        BP.SELECCION('PROVINCIA')          #Filtra por provincia
        prov = BP.nombres()                #Extrae los nombres de provincia
        filas1 = BP.indices()              #Extrae los códigos de provincias
        for z,p in zip(filas1,prov):       #Itera para cada provincia
            BP.SELECCION('PROVINCIA',z)    #Selecciona provincia específica
            BP.SELECCION('MUNICIPALIDAD')  #Filtra opor municipalidad
            mun = BP.nombres()             #Extrae nombres de las municipalidades
            filas2 = BP.indices()          #Extrae codigos de municipalidaddes
            for t,v in zip(filas2,mun):    #Itera para cada municipalidad
                BP.SELECCION('MUNICIPALIDAD',t) #Selecciona una municipalidad espcífica
                #Extrae datos de proyectos agregando columnas de departamento, provincia y municipalidad
                contenedor.append(BP.SELECCION('PROYECTO','TODO_A',{'DEPTO':x, 'Nomb_dep':y,
                                                                    'PROV':z, 'Nomb_prov':p,
                                                                    'MUNI':t, 'Nomb_muni':v}))
                BP.retroceder() #Regresa a la ventana anterior
            BP.retroceder()     #Regresa a la aventana anterior
        BP.retroceder()         #Regresa a la ventana anterior
    BP.CERRAR()                            #Cierra el navegador al terminar la iteración 
    proy_inversion = pd.concat(contenedor) #Se concatenan las tablas contenidas en la lista
    proy_inversion = proy_inversion.reset_index(drop=True)
    return proy_inversion

In [ ]:
start = time.time()

#Extrae los datos de de proyectos del 2004 al 2022 (realiza cuatro búsquedas al mismo tiempo)
GR = Parallel(n_jobs=4)(delayed(CONSULTA_GL)(x) for x in tqdm(range(2004,2022)))

end = time.time()
print(end - start)

### **2. Sistema de Seguimiento de Inversiones**

In [ ]:
import pandas as pd
import Scrapper_SSI #Automatiza la descarga de datos en el Sistema de Seguimiento de Inversiones 
from Scrapper_SSI import INVERSIONES
from tqdm import tqdm #Muestra la barra de progreso de una iteración 
from tqdm.notebook import tqdm_notebook

In [ ]:
def agrupar(a, n):
    '''
    Funcion para separar lista de codigos en grupos más asequibles
    '''
    k, m = divmod(len(a), n)
    return [a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]

In [ ]:
GL= pd.read_excel('GL_CODES.xlsx')
listas = agrupar(list(GL['cod']),20)
len(listas[1]) #Divide las listas en 20 grupos

In [ ]:
def araña(P = []):
    '''
    Función de extracción de datos del SSI
    '''
    import Scrapper_SSI #Automatiza la descarga de datos en el Sistema de Seguimiento de Inversiones 
    from Scrapper_SSI import INVERSIONES
    from tqdm import tqdm #Muestra la barra de progreso de una iteración 
    from tqdm.notebook import tqdm_notebook 
    
    SI= INVERSIONES() #Abre el navegador de Firefox y se redirige a la página del SSI
    contenido = []    #Creamos una lista vacía donde guardar los datos extraídos
    for x in tqdm(P): #Iteramos para cada código de inversión de la serie 
        SI.BUSCAR(x)  #Busca el código en el SSI
        contenido.append(SI.COMPLETO_RES()) #Guarda la información en la lista vacia "contenido"
    SI.CERRAR()       #Cierra el navegador al terminar la iteración 
    proy = pd.concat(contenido) #Se concatenan las tablas contenidas en la lista
    return proy

In [ ]:
from joblib import Parallel, delayed #Permite paralelizar los test de selenium 
import time

start = time.time()
proyectos = Parallel(n_jobs=5)(delayed(araña)(x) for x in tqdm_notebook(listas,desc='Progreso'))

end = time.time()
print(end - start)

In [ ]:
GL = pd.concat(proyectos)
GL.to_pickle('GL14.pkl') # Guardamos en formato pickle
GL.to_pickle('GL14.pkl') # Guardamos en formato excel